In [3]:
import cv2
import numpy as np

def preprocess_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to remove noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply thresholding to segment the number plate
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return thresh

def extract_number_plate(image):
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    
    # Apply morphological operations to further enhance the number plate region
    kernel = np.ones((5, 5), np.uint8)
    morphed = cv2.morphologyEx(preprocessed_image, cv2.MORPH_CLOSE, kernel)
    
    # Find contours in the image
    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours based on their area
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]  # Assuming largest contour is the number plate
    
    # Extract the bounding box of the number plate
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        number_plate_roi = image[y:y+h, x:x+w]
        
        # Display the bounding box
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        return number_plate_roi

# Load the image
image = cv2.imread('download.jpg')

# Extract the number plate
number_plate = extract_number_plate(image)

# Display the results
cv2.imshow('Number Plate', number_plate)
cv2.waitKey(0)
cv2.destroyAllWindows()
